In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# from custom_matrix import Matrix, IdentityMatrix
# from custom_matrix import simplemul, matmul, transpose, matadd, matsub

# m1 = Matrix(3, 4, data=[2, 6, 9, 5, 8, 9, 4, 3, 2, 4, 3, 8])
# m2 = Matrix(4, 3, data=[8, 7, 6, 5, 9, 4, 3, 3, 2, 8, 1, 0])
# i = IdentityMatrix(3)
# print(i.rows)

# m = matmul(m1, m2)
# print(m.rows)
# m = matmul(m, i)
# print(m.rows)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[113, 100, 54], [145, 152, 92], [109, 67, 34]]
[[113, 100, 54], [145, 152, 92], [109, 67, 34]]


In [2]:
import pandas as pd
import numpy as np
import fitparse

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

# activity_file = '../../buttes.fit'
# activity_file = '../../canal.fit'
# activity_file = '../../montmartre.fit'
# activity_file = '../../cotes.fit'
# activity_file = '../../seuil.fit'
# activity_file = '../../libourne.fit'
# activity_file = '../../velo.fit'
# activity_file = '../../vma.fit'
activity_file = '../../10km_rouen.fit'

fitfile = fitparse.FitFile(activity_file)

session_raw = pd.DataFrame([record.get_values() for record in fitfile.get_messages("record")])
session = session_raw[['timestamp', 'distance', 'enhanced_speed', 'enhanced_altitude', 'heart_rate', 'power', 'GAP', 'Grade']]
start_time = session.timestamp.values[0]
session['time'] = np.arange(session.shape[0])
session.head(20)

,timestamp,distance,enhanced_speed,enhanced_altitude,heart_rate,power,GAP,Grade,time
0,2022-09-24 18:00:10,0.00,0.000,8.6,137,167,0.000000,0.000000,0
1,2022-09-24 18:00:11,4.00,0.000,8.6,138,179,0.000000,0.000000,1
2,2022-09-24 18:00:12,7.58,1.381,8.6,140,186,0.000000,0.000000,2
3,2022-09-24 18:00:13,12.11,3.200,8.6,144,342,0.000000,0.000000,3
4,2022-09-24 18:00:14,16.29,3.424,8.6,145,401,0.000000,0.000000,4
5,2022-09-24 18:00:15,20.39,3.919,8.6,148,458,0.000000,0.000000,5
6,2022-09-24 18:00:16,23.66,3.919,8.6,149,465,14.108006,0.000000,6
7,2022-09-24 18:00:17,27.43,3.919,8.6,151,490,14.108006,0.000000,7
8,2022-09-24 18:00:18,31.51,3.620,8.6,152,431,14.108006,0.000000,8
9,2022-09-24 18:00:19,35.37,3.620,8.8,154,408,13.033110,0.000000,9


In [6]:
run = pd.read_csv('../../10km_rouen_data.csv')
mapping = {
    'record.distance[m]': 'distance', 
    'record.enhanced_altitude[m]': 'altitude', 
    'record.enhanced_speed[m/s]': 'speed', 
    'record.developer.0.GAP': 'gap', 
    'record.power[watts]': 'power', 
    'record.heart_rate[bpm]': 'heart_rate', 
    'record.developer.0.GAP efficiency factor': 'efficiency', 
    'record.cadence[rpm]': 'cadence', 
    'record.developer.0.Grade': 'grade',
}
usecols = [c for c in mapping.keys() if c in run.columns]
run['time'] = np.arange(run.shape[0])
run = run[['time'] + usecols].rename(columns=mapping)
run.head(20)

,time,distance,altitude,speed,gap,power,heart_rate,efficiency,cadence,grade
0,0,0.00,8.6,0.000,0.000000,167,137,0.000000,90,0.000000
1,1,4.00,8.6,0.000,0.000000,179,138,0.000000,90,0.000000
2,2,7.58,8.6,1.381,0.000000,186,140,0.000000,92,0.000000
3,3,12.11,8.6,3.200,0.000000,342,144,0.000000,93,0.000000
4,4,16.29,8.6,3.424,0.000000,401,145,0.000000,94,0.000000
5,5,20.39,8.6,3.919,0.000000,458,148,0.000000,95,0.000000
6,6,23.66,8.6,3.919,14.108006,465,149,1.588739,95,0.000000
7,7,27.43,8.6,3.919,14.108006,490,151,1.578077,94,0.000000
8,8,31.51,8.6,3.620,14.108006,431,152,1.571047,93,0.000000
9,9,35.37,8.8,3.620,13.033110,408,154,1.535139,93,0.000000


In [7]:
fig = go.Figure()

# fig.add_trace(go.Scatter(x=run.time, y=run.distance, mode='lines', name='distance'))

# fig.add_trace(go.Scatter(x=run.time, y=run.speed, mode='lines', name='speed'))
# fig.add_trace(go.Scatter(x=run.time, y=run.gap, mode='lines', name='gap'))
# fig.add_trace(go.Scatter(x=run.time, y=run.power, mode='lines', name='power'))

fig.add_trace(go.Scatter(x=run.time, y=run.altitude, mode='lines', name='altitude'))
fig.add_trace(go.Scatter(x=run.time, y=run.grade, mode='lines', name='grade'))

# fig.add_trace(go.Scatter(x=run.time, y=run.heart_rate, mode='lines', name='heart_rate'))
# fig.add_trace(go.Scatter(x=run.time, y=run.cadence, mode='lines', name='cadence'))

fig.update_layout(width=2000, height=800)

fig.show()

## 1. Kalman filters

In [26]:
from kalman import KalmanFilter1D
from kalman import KalmanFilter

# Raw speed
# session['speed'] = session.distance.values - session.distance.shift(1).values

# Filtered features
# filtered_speed = KalmanFilter1D(2, 0.5, 1, 0.1)
# session['filtered_speed1'] = [filtered_speed.get_current_estimate(m) for m in session.speed]
# filtered_speed = KalmanFilter1D(2, 0.5, 1, 0.15)
# session['filtered_speed2'] = [filtered_speed.get_current_estimate(m) for m in session.speed]

## BEST ##
# power is already a derived metric, so the 1d is well designed to properly estimate it
# filtered_power_1d = KalmanFilter1D(100, 1, 3, 1)
# session['filtered_power_1d'] = [filtered_power_1d.get_current_estimate(m) for m in session.power]

# filtered_power = KalmanFilter([100, 10], 1, 10, 0.05)
# session['filtered_power'] = [filtered_power.get_current_estimate(m) for m in session.power]

# filtered_altitude = KalmanFilter1D(session.enhanced_altitude[0], 1, 2, 1.5)
# session['filtered_altitude_1d'] = [filtered_altitude.get_current_estimate(m) for m in session.enhanced_altitude]

filtered_altitude = KalmanFilter([session.enhanced_altitude[0], 0], 1, 2, 0.05)
session['filtered_altitude'] = [filtered_altitude.get_current_estimate(m) for m in session.enhanced_altitude]


# Rolling features
rolling_period = 15
# session['rolling_speed'] = session.speed.rolling(rolling_period).mean()
# session['rolling_power'] = session.power.rolling(rolling_period).mean()
session['rolling_altitude'] = session.enhanced_altitude.rolling(rolling_period).mean()

# plt.figure(figsize=(20, 8))
# # plt.plot(session.time, session.enhanced_altitude, label='enhanced_altitude')
# plt.plot(session.time, session.power, label='power')
# plt.plot(session.time, session.rolling_power, label='rolling_power')
# # plt.plot(session.time, session.enhanced_speed, label='enhanced_speed')
# # plt.plot(session.time, session.speed, label='speed')
# plt.legend()
# plt.show()

fig = go.Figure()

# fig.add_trace(go.Scatter(x=session.time, y=session.power, mode='lines', name='power'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_power, mode='lines', name='rolling power'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_power_1d, mode='lines', name='filtered power 1d'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_power, mode='lines', name='filtered power'))

fig.add_trace(go.Scatter(x=session.time, y=session.enhanced_altitude, mode='lines', name='altitude'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_altitude, mode='lines', name='rolling altitude'))
# # fig.add_trace(go.Scatter(x=session.time, y=session.filtered_altitude_1d, mode='lines', name='filtered altitude 1d'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_altitude, mode='lines', name='filtered altitude'))

# fig.add_trace(go.Scatter(x=session.time, y=session.speed, mode='lines', name='speed'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_speed, mode='lines', name='rolling speed'))
# fig.add_trace(go.Scatter(x=session.time, y=session.enhanced_speed, mode='lines', name='enhanced speed'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_speed1, mode='lines', name='filtered speed 1'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_speed2, mode='lines', name='filtered speed 2'))


fig.update_layout(width=1700, height=800)

fig.show()

## 2. Grade estimation

In [27]:
from custom_queue import Queue

def compute_grade_legacy(session, rolling_period):

    grade = []

    altitude = Queue(rolling_period+1)
    speed = Queue(rolling_period+1)

    for s in range(session.shape[0]):
        
        altitude.update(session.enhanced_altitude.values[s])
        speed.update(session.enhanced_speed.values[s])

        spd = speed.mean()
        n_seconds = altitude.count_not_null()
        dist = spd * n_seconds
        
        g = 100 * (altitude.current(2) - altitude.last(2)) / dist if ((n_seconds >= 5) & (dist > 1)) else 0
        g = 40 if (g > 40) else g
        g = -40 if (g < -40) else g

        grade.append(g)

    return grade

session['estimated_grade'] = compute_grade_legacy(session, 15)

In [28]:
filtered_grade = KalmanFilter1D(0, 1, 0.5, 0.1)
session['filtered_grade_1d'] = [filtered_grade.get_current_estimate(m) for m in session.estimated_grade]

# filtered_grade = KalmanFilter([0, 0], 1, 1.5, 0.05)
# session['filtered_grade'] = [filtered_grade.get_current_estimate(m) for m in session.estimated_grade]
# session['filtered_grade_spe'] = session.filtered_grade.map(lambda g: g**3 if (g > -1) & (g < 1) else g)

session['rolling_grade'] = session.estimated_grade.rolling(5).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(x=session.time, y=session.enhanced_altitude, mode='lines', name='altitude'))
fig.add_trace(go.Scatter(x=session.time, y=session.estimated_grade, mode='lines', name='estimated_grade'))

# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_grade_1d, mode='lines', name='filtered_grade_1d'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_grade, mode='lines', name='filtered_grade'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_grade_spe, mode='lines', name='filtered_grade_spe'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_grade, mode='lines', name='rolling_grade'))

fig.update_layout(width=1700, height=800)

fig.show()